# Support Vector Machines

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Exploración del Dataset

In [2]:
## nos vamos a quedar solo con dos caracteristicas, los sepalos, 
## para poder representarlo en un grafico de dos ejes ya que este ejercicio tiene finalidad didactica

XC = "Sepal.Length"
YC = "Sepal.Width"
TC = "Species"

data = pd.read_csv("datasets/iris.csv")[[XC, YC, TC]]

print(data.shape)
data.head()

(150, 3)


,Petal.Length,Sepal.Width,Species
0,1.4,3.5,setosa
1,1.4,3.0,setosa
2,1.3,3.2,setosa
3,1.5,3.1,setosa
4,1.4,3.6,setosa


## Conjuntos de entrenamiento y prueba

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size = 0.2, random_state = 50)

X_train = train[[XC, YC]]
Y_train = train[TC].replace("virginica", 0).replace("versicolor", 1).replace("setosa", 2)

X_test = test[[XC, YC]]
Y_test = test[TC].replace("virginica", 0).replace("versicolor", 1).replace("setosa", 2)

## Modelo SVM

In [4]:
from sklearn import svm

In [5]:
# hacemos un modelo linear y otro radial para comparar resultado con distintos kernels
C = 1.0
SVM_linear = svm.SVC(kernel="linear", C=C, decision_function_shape="ovr").fit(X_train,Y_train) #ovr ajuste de datos lo mejor posible
SVM_radial = svm.SVC(kernel="rbf", C=C, decision_function_shape="ovr").fit(X_train,Y_train) #ovr ajuste de datos lo mejor posible

### Realizamos algunas predicciones

In [6]:
df = X_test.copy()
df["real"] = Y_test
df["pred"] = SVM_linear.predict(X_test)

df.head()

,Petal.Length,Sepal.Width,real,pred
88,4.1,3.0,1,1
72,4.9,2.5,1,0
20,1.7,3.4,2,2
16,1.3,3.9,2,2
147,5.2,3.0,0,0


## Selección de parametros óptimos

Utilizando la técnica de cross-validation

In [7]:
## veamos cual es mejor usando la tecnica del grid cross validation

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils import shuffle

In [8]:
parameters = [
    { 
        'kernel': ["rbf"],
        'gamma': [1e-4,1e-3,1e-2,0.1,0.2,0.5],
        'C': [1,10,100,1000]
    },
    {
        'kernel': ["linear"],
        'C': [1,10,100,1000]
    }
]

clf = GridSearchCV(svm.SVC(decision_function_shape="ovr"), param_grid=parameters, cv=5)
clf.fit(X_train,Y_train)

print("Los mejores parametros son: ", str(clf.best_params_))

Los mejores parametros son:  {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}


### Evaluación del modelo

In [9]:
medias = clf.cv_results_["mean_test_score"]
desviaciones_estandar = clf.cv_results_["std_test_score"]
parametros = clf.cv_results_["params"]

for m, s, p in zip(medias, desviaciones_estandar, parametros):
    print("Puntuación media: %0.3f (+/-%0.2f) para %r"%(m,s,p))

Puntuación media: 0.533 (+/-0.16) para {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
Puntuación media: 0.683 (+/-0.02) para {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Puntuación media: 0.950 (+/-0.03) para {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Puntuación media: 0.958 (+/-0.05) para {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Puntuación media: 0.958 (+/-0.05) para {'C': 1, 'gamma': 0.2, 'kernel': 'rbf'}
Puntuación media: 0.958 (+/-0.05) para {'C': 1, 'gamma': 0.5, 'kernel': 'rbf'}
Puntuación media: 0.683 (+/-0.02) para {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
Puntuación media: 0.950 (+/-0.03) para {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Puntuación media: 0.958 (+/-0.05) para {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Puntuación media: 0.958 (+/-0.05) para {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Puntuación media: 0.958 (+/-0.05) para {'C': 10, 'gamma': 0.2, 'kernel': 'rbf'}
Puntuación media: 0.958 (+/-0.05) para {'C': 10, 'gamma': 0.5, 'kernel': 'rbf'}
Puntuación media: 0.950 (+/-0.03) 

### Test con los parametros óptimos

In [10]:
Y_pred = clf.predict(X_test)

print(classification_report(Y_test, Y_pred, target_names=["virginica", "versicolor", "setosa"]))

# Recordatorio:
## precision = especifidad (verdaderos aciertos)
## recall = sensibilidad (falsos negativos)

              precision    recall  f1-score   support

   virginica       0.82      1.00      0.90         9
  versicolor       1.00      0.83      0.91        12
      setosa       1.00      1.00      1.00         9

    accuracy                           0.93        30
   macro avg       0.94      0.94      0.94        30
weighted avg       0.95      0.93      0.93        30



## Representación gráfica

De manera interactiva.

In [11]:
def svm_visualize(X,Y, C=1.0, gamma=0.01, kernel="rbf"):
    
    import pandas as pd
    import numpy as np
    from sklearn import svm
    import matplotlib.pyplot as plt
    
    x_min, x_max = X.iloc[:,0].min()-1, X.iloc[:,0].max()+1
    y_min, y_max = X.iloc[:,1].min()-1, X.iloc[:,1].max()+1
    h = (x_max - x_min)/100 #dividimos para tener 100 trocitos
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h)) 
    X_plot = np.c_[xx.ravel(), yy.ravel()]

    svc = svm.SVC(kernel=kernel, C=C, gamma=gamma, decision_function_shape="ovr").fit(X,Y) #ovr ajuste de datos lo mejor posible
    Ypred = svc.predict(X_plot)
    Ypred = Ypred.reshape(xx.shape)
    
    plt.figure(figsize=(15,5))
    plt.contourf(xx,yy,Ypred, cmap=plt.cm.tab10, alpha = 0.3) #(tab10 son 10 colores con transparencias)
    plt.scatter(X.iloc[:,0], X.iloc[:,1], c=Y, cmap=plt.cm.Set1)

    plt.xlim(xx.min(), xx.max())
    
    plt.title("SVM, kernel " + kernel)

In [12]:
from ipywidgets import interact, fixed
from sklearn import datasets

X = data[[XC,YC]]
Y = data[TC].replace("virginica", 0).replace("versicolor", 1).replace("setosa", 2)
    
interact(svm_visualize, X=fixed(X), Y=fixed(Y),
         C=[0.01, 0.1, 1,10,100,1000,1e6, 1e10], 
         gamma=[1e-5,1e-4,1e-3,1e-2,0.1,0.2,0.5,0.99],
         kernel=["rbf","linear","sigmoid","poly","precomputed"])

interactive(children=(Dropdown(description='C', index=2, options=(0.01, 0.1, 1, 10, 100, 1000, 1000000.0, 1000…

<function __main__.svm_visualize(X, Y, C=1.0, gamma=0.01, kernel='rbf')>